In [ ]:
from data_classes.TextLightningDataModule import TextLightningDataModule
from models.ClassifierSystem import LightningClassifier
from data_classes.pretrained_embeddings import get_pretrained_embeddings
from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [ ]:
vocab, vectors = get_pretrained_embeddings()

In [ ]:
imdb_data = TextLightningDataModule(vocab, dataset="IMDB")

In [ ]:
classifier = LightningClassifier(num_class=2, vocab=vocab, vectors=vectors, embedding_size=300, learning_rate=1e-3, num_layers=1, hidden_size=100)

In [ ]:
logger_tensor = TensorBoardLogger('exp_logs', name='lstm')
logger_csv = CSVLogger("csv_logs", name="exp_csv")

In [ ]:
trainer = Trainer(max_epochs=10, gpus=1, auto_select_gpus=True, callbacks=[EarlyStopping(monitor="Val Loss", patience=3)], logger=[logger_tensor, logger_csv])
# , track_grad_norm=2, accumulate_grad_batches=8, logger=[logger]

In [ ]:
# Check everything is working here:
# for rnn in ["RNN", "LSTM", "GRU"]:
#     for att in [None, "last_hidden_layer", "self"]:
#         for output_layer_type in ["linear", "MLP"]:
#             for num_layers in [1, 2]:
#                 for hidden_size in [100, 300]:
#                     classifier = LightningClassifier(num_class=5, vocab=vocab, vectors=vectors, embedding_size=300, learning_rate=1e-3, num_layers=num_layers, hidden_size=hidden_size, model_type=rnn, attention_type=att, output_layer_type=output_layer_type)
#                     trainer.fit(classifier, imdb_data)


In [ ]:
# import torch
# for batch in imdb_data.val_dataloader():
#     x, y, l = batch
#     print(y)
#     y_pred = classifier.model(x.to("cpu"), l.to("cpu"))
#     print(y_pred)
#     loss = torch.nn.CrossEntropyLoss()(y_pred, y)
#     labels_hat = torch.argmax(y_pred, dim=1)
#     test_acc = torch.sum(labels_hat == y).item() / (len(y) * 1.0)
#     break

In [ ]:
trainer.test(classifier, imdb_data)

In [ ]:
# trainer.tune(classifier, imdb_data)

In [ ]:
# classifier.batch_size

In [ ]:
classifier.learning_rate

In [ ]:
trainer.fit(classifier, imdb_data)

In [ ]:
trainer.test(classifier, imdb_data)

In [ ]:
# Run learning rate finder
# lr_finder = trainer.tuner.lr_find(classifier, imdb_data)

In [ ]:
# Plot with
# fig = lr_finder.plot(suggest=True)
# fig.show()

In [ ]:
# Pick point based on plot, or get suggestion
# new_lr = lr_finder.suggestion()
# new_lr

In [ ]:
import torch
import torchmetrics.functional as F

preds = torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])
target = torch.tensor([0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1])

print(F.confusion_matrix(preds, target, num_classes=2))
print(F.precision_recall(preds, target, num_classes=2, multiclass=True))
print(F.accuracy(preds, target, num_classes=2))

In [ ]:
F.confusion_matrix(preds, target, num_classes=2)